In [1]:
import pandas as pd
import json
import re
from fractions import Fraction
import typing
import os
import ast
from googletrans import Translator, constants

filename = '/Users/Ian van de Wetering/Documents/TUe/Knowledge Engineering 2AMD20/dataset/full_dataset.csv'

df_full = pd.read_csv(filename)

# No need to check and remove Null Values, because the following code that there are no null values
# null_values = df_full.isnull().sum()
# print(null_values)

columns_to_remove = ['directions', 'link', 'source']
df_full = df_full.drop(columns_to_remove, axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df: pd.DataFrame = pd.DataFrame(df_full[:100])
#print(df)

import typing
unitConversions: dict = {
    'c ': '236.588 ml',
    'c.': '236.588 ml',
    'tsp.': '4.92892 ml',
    'tbsp.': '14.7868 ml',
    'oz.': '29.5735 ml',
    'pt.': '473.176 ml',
    'qt.':' 946.353 ml',
    'lb.': '453.592 grams',
    'gal.': '3785.41 ml',
    ' oz': ' 29.5735 ml',
}

def replace_units(text: str) -> str:
    # replaces american units with european units
    text= text.lower()
    for key, value in unitConversions.items():
        text = text.replace(key, value)
    return text
def replace_fractions(text:str)-> str:
    # replaces fractions with decimals
    text = text.lower()
    text = re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, text)
    return text
def frac2string(s):
    # helper function for replace_fractions
    i, f = s.groups(0)
    f = Fraction(f)
    return str(int(i) + float(f))
# Function to simplify ingredients by only keeping the last word 
def keep_last_word(item):
    words = [x.split()[-1] for x in eval(item)]
    return str(words)
# Function to replace list items based on keyword list
def replace_list_items(lst):
    result = []
    my_list = lst.split(', ')
    for item in my_list:
        item = item[1:-1] # Remove square brackets in the beginning or in the end
        if item[0] == '"':
            item = item[1:]
        if item.endswith('"'): 
            item = item[:-1] # Remove double quotes to get clean strings
        for keyword in keywords:
            if keyword in item:
                item = keyword
                break
        result.append(item)
    return str(result)

# This function cleans up the units of the column ingredients and translate
# the ingredients into g , ml, or stuk 

def get_units(lst):
    result = []
    for item in lst:
        if "grams" in item:
            result.append("g")
        elif "ml" in item:
            result.append("ml")
        else :
            result.append("stuk")    
    return str(result)

# This function uses the column ingredient to find
# the exact amount of ingredients used by checking
# if the first part of the string is a digit 
def get_amounts(lst):
    result = []
    for item in lst:
        if item.split()[0][0].isdigit():
            result.append(float(item.split()[0]))
        else:
            result.append(float(1.0))
          
    return result

# This function translates all the units in the pricing to units that can be used
# for calculation
def get_units_simplified(lst):
    result = []
    
    for item in lst:
        if "NaN" in item:
            result.append("NaN")
        elif "kg" in item:
            result.append("kg")
        elif "g" in item:
            result.append("g")
        elif "stuk" in item:
            result.append("stuk")
        elif "ml" in item:
            result.append("ml")
        elif "l" in item:
            result.append("l")
        else:
            result.append("stuk")
        
    return str(result)

# This function removes NaN and replaces it by 1.0 and also transforms Dutch floats with commas to English floats with point
def get_quantity_simplified(lst):
    result = []
    for item in lst:
        #print(item.split())
        if "," in item.split()[0]:
            result.append(float(item.split()[0][:1] + "." + item.split()[0][2:]))
        elif item.split()[0][0].isdigit():
            result.append(float(item.split()[0]))
        elif item.split()[0] == "NaN":
            result.append(float(1.0))
        else:
            result.append(float(1.0))
    return result

# This function replaces NaN pricing with 0.0 so that the calculations can be executed for all items
# being a float, because the price is set to 0 it will not be added to the total price
def get_price_simplified(lst):
    result = []
    
    for item in lst:
        if item == "NaN":
            result.append(float(0.0))
        else:
            result.append(item)
    return result

# This function calculates the total price of the recipe by multiplying price with used amounts divided by price of the amount 
# if the unit of the price is l or kg, the price is divided by 1000 to reflect pricing per ml or gram 
# The total price is calculated by summing all the individual ingredients of the recipe
def calculate_price_recipe(lst1, lst2, lst3, lst4, lst5):
    result = []
    res = [a * (x / y) / 1000 if (z =="'l'," or z=="'kg'," ) else a * (x / y) for x, y, z, a in zip(lst1, lst3, lst4.split(), lst5)]
    result = sum(res)
    return result

def convert_and_multiply_units(text: str) -> list:
    # converts units to floats within strings and multiplies them
    # e.g 1 0.5 0.24 ml becomes 0.12 ml
    ingredients: list = json.loads(text)
    ingredient_list = []
    for ingredient in ingredients: # ingredient is a string with units and one specific ingredient
        ingredient_split: list[str] = re.split(r'\s+|\)|\(', ingredient)
        filtered_result = [item for item in ingredient_split if item]
        converted_values: list= []
        for index, item in enumerate(filtered_result):
            try: # try to turn it into a float and multiply
                converted_value = float(item)
                if len(converted_values) >= 1 and isinstance(converted_values[-1], float):
                    converted_value = converted_values[-1] * converted_value # multiply latest float with current
                    converted_values.pop(-1) # remove latest value, since were using it to multiply
                    converted_values.append(float(converted_value)) 
                else:
                    converted_values.append(converted_value)   
            except ValueError:
                converted_values.append(item)
                pass
            except  IndexError:
                pass
        ingredient_string = " ".join(str(item) for item in converted_values) # join the list back together    
        ingredient_list.append(ingredient_string)
    return ingredient_list


df['ingredients'] = df['ingredients'].apply(lambda text: replace_units(text))
df['ingredients'] = df['ingredients'].apply(lambda text: replace_fractions(text))
df['ingredients'] = df['ingredients'].apply(lambda text: convert_and_multiply_units(text))
#df['NER_simple'] = df['NER'].apply(keep_last_word) #This algorithm is too simple, in stead a mapping function will be used using an extra database

recipe_df=df

# helper functions
def translator(output_path='ah_products_en.csv', write=True) -> pd.DataFrame:
  # translates if file does not exist
    if os.path.isfile(output_path): # if file exists skip
        print(f'File {output_path} already exists, skipping translation')
        ah_products = pd.read_csv(output_path)
    else:
        supermarket_df = pd.read_json('supermarket.json', encoding='UTF-8')
        ah_products = pd.DataFrame(supermarket_df.iloc[0]['d'])
        translator = Translator()
        tarray = []

        for product in ah_products['n']:
            translations = translator.translate(product, src="nl", dest="en")
            tarray.append(translations.text)

        ah_products['l_en'] = tarray
        if write:
            ah_products.to_csv(output_path)
            print(f'Wrote translated products to {output_path}')
        else:
            print("Done translating inplace")
    return ah_products

def product_price_quantity_returner_per_ingredient(product: str, supermarket_df: pd.DataFrame) -> int:
  # Returns the closest product name, price and quantity when given a product string
    candidate_products = supermarket_df[supermarket_df['l_en'].str.contains(f'{product}')]
    if len(candidate_products) == 0:
        print(f'No products found for {product}')
        return 'NaN', 'NaN', 'NaN'  
    min_length_idx = candidate_products['l_en'].str.len().idxmin()
    # Use this index to get the shortest string
    return candidate_products.loc[min_length_idx, 'l_en'], candidate_products.loc[min_length_idx, 'p'], candidate_products.loc[min_length_idx, 's']

def mass_recipe_converter(text: list, products_df: pd.DataFrame) -> list:
    text = text.replace('+', 'NaN') # this is because literal_eval cannot handle + in strings, it's a regex thing
    simple_ingredients: list = ast.literal_eval(text)
    prices = []
    quantities = []
    products = []
    # for each SIMPLE ingredient, find the AH product translation and its price and append it here
    for ingredient in simple_ingredients:
        product, price, quantity = product_price_quantity_returner_per_ingredient(ingredient, products_df)
        prices.append(price)
        quantities.append(quantity)
        products.append(product)
    return products, prices, quantities

# Read the pkl file from https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=ingr_map.pkl
unpickled_df = pd.read_pickle("ingr_map.pkl")

# Create a list of unique ingredients of the ingr_map.pkl file
keywords = unpickled_df['replaced'].unique().tolist()
# Clean a few obvious items that will disturb the mapping 
keywords = [item.rstrip("'") for item in keywords]
keywords = [item for item in keywords if len(item) != 1]
#append_list = ["water", "sugar"]
#for item in append_list:
    #keywords.append(item)
recipe_df['NER_mapped'] = recipe_df['NER'].apply(replace_list_items)
    
translated_products = translator()
triplet_NER = recipe_df['NER_mapped'].apply(lambda text: mass_recipe_converter(text, translated_products))
recipe_df['NER_product'], recipe_df['NER_price'], recipe_df['NER_quantity'] = zip(*triplet_NER)

# -------THIS Section calculates the total price of the recipe assuming
# correctness of the previous columns ----------------------------

# This command calculates the exact amount of ingredients 
# used from the column ingredients and stores it
# in a new column named "NER_amount_used"
# Note that the quality of column ingredients seems not of the best quality
recipe_df['NER_amount_used'] = recipe_df['ingredients'].apply(get_amounts)
# This command calculates the exact amount of units used by the recipe but 
# and simplifies it into g, ml and stuk based on the column ingredients
recipe_df['NER_unit_used'] = recipe_df['ingredients'].apply(get_units)
# These next 2 commands calculate the exact amount and unit to which the price is applicable 
# For example, 0.5 l costs 1.29 euros will result in 0.5 as the amount and "l" as the unit
recipe_df['NER_price_amount'] = recipe_df['NER_quantity'].apply(get_quantity_simplified)
recipe_df['NER_price_unit'] = recipe_df['NER_quantity'].apply(get_units_simplified)
# Simplify price in English by removing setting the NaN to zero for calculation purposes
# and by replacing , into . for English calculations
# also if the related unit is l or kg, the price is divided by 1000 to reflect pricing per ml and per gram
recipe_df['NER_price_simplified'] = recipe_df['NER_price'].apply(get_price_simplified)
# Iterate over all items of the dataframe
recipe_df['NER_recipe_total_price'] = recipe_df.apply(lambda row: calculate_price_recipe(row['NER_amount_used'], row['NER_unit_used'], row['NER_price_amount'], row['NER_price_unit'], row['NER_price_simplified']), axis=1)

recipe_df.head(100)


File ah_products_en.csv already exists, skipping translation
No products found for brown sugar
No products found for cream of mushroom soup
No products found for garlic powder
No products found for chicken gravy
No products found for cream of mushroom soup
No products found for baking potato
No products found for Worcestershire sauce
No products found for condensed milk
No products found for graham cracker crust
No products found for dark sweet pitted cherries
No products found for ginger ale
No products found for boiling water
No products found for almond extract
No products found for paraffin
No products found for pie filling
No products found for condensed milk
No products found for cleaned strawberries
No products found for white cake
No products found for frozen strawberries
No products found for boiling water
No products found for shortening
No products found for shortening
No products found for Frango
No products found for tomato paste
No products found for ground black pepper
N

,Unnamed: 0,title,ingredients,NER,NER_mapped,NER_product,NER_price,NER_quantity,NER_amount_used,NER_unit_used,NER_price_amount,NER_price_unit,NER_price_simplified,NER_recipe_total_price
0,0,No-Bake Nut Cookies,"[236.588 ml firmly packed brown sugar, 118.294 ml evaporated milk, 2.46446 ml vanilla, 118.294 ml broken nuts pecans, 29.5736 ml butter or margarine, 828.058 ml bite size shredded rice biscuits]","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""butter"", ""bite size shredded rice biscuits""]","['brown sugar', 'milk', 'vanilla', 'nut', 'butter', 'rice']","[NaN, AH Buttermilk, AH Muffin vanilla, AH Coconut, AH Herb butter, AH Sushi rice]","[NaN, 0.85, 1.66, 2.0, 1.25, 2.49]","[NaN, 0,5 l, 300 g, 90 g, 100 g, 500 g]","[236.588, 118.294, 2.46446, 118.294, 29.5736, 828.058]","['ml', 'ml', 'ml', 'ml', 'ml', 'ml']","[1.0, 0.5, 300.0, 90.0, 100.0, 500.0]","['NaN', 'l', 'g', 'g', 'g', 'g']","[0.0, 0.85, 1.66, 2.0, 1.25, 2.49]",7.336891
1,1,Jewell Ball'S Chicken,"[1.0 small jar chipped beef, cut up, 4.0 boned chicken breasts, 1.0 can cream of mushroom soup, 1.0 carton sour cream]","[""beef"", ""chicken breasts"", ""cream of mushroom soup"", ""sour cream""]","['beef', 'chicken breast', 'cream of mushroom soup', 'sour cream']","[AH roast beef, Encore chicken breast, NaN, AH Oat sour cream]","[2.99, 1.29, NaN, 0.95]","[100 g, 70 g, NaN, 120 g]","[1.0, 4.0, 1.0, 1.0]","['stuk', 'stuk', 'stuk', 'stuk']","[100.0, 70.0, 1.0, 120.0]","['g', 'g', 'NaN', 'g']","[2.99, 1.29, 0.0, 0.95]",0.111531
2,2,Creamy Corn,"[946.352 ml pkg. frozen corn, 236.588 ml pkg. cream cheese, cubed, 78.86266666666666 ml butter, cubed, 2.46446 ml garli236.588 mlpowder, 2.46446 ml salt, 1.23223 ml pepper]","[""frozen corn"", ""cream cheese"", ""butter"", ""garlic powder"", ""salt"", ""pepper""]","['corn', 'cream cheese', 'butter', 'garlic powder', 'salt', 'pepper']","[AH Popcorn Salt, AH Soft herb cream cheese, AH Herb butter, NaN, AH Peas 0% salt, AH Red pepper]","[1.05, 0.85, 1.25, NaN, 1.69, 0.44]","[100 g, 125 g, 100 g, NaN, 680 g, per stuk]","[946.352, 236.588, 78.86266666666666, 2.46446, 2.46446, 1.23223]","['ml', 'ml', 'ml', 'ml', 'ml', 'ml']","[100.0, 125.0, 100.0, 1.0, 680.0, 1.0]","['g', 'g', 'g', 'NaN', 'g', 'stuk']","[1.05, 0.85, 1.25, 0.0, 1.69, 0.44]",13.079584
3,3,Chicken Funny,"[1.0 large whole chicken, 621.0435 ml cans chicken gravy, 310.52175 ml can cream of mushroom soup, 177.441 ml box stove top stuffing, 118.294 ml shredded cheese]","[""chicken"", ""chicken gravy"", ""cream of mushroom soup"", ""shredded cheese""]","['chicken', 'chicken gravy', 'cream of mushroom soup', 'cheese']","[AH Bapao chicken, NaN, NaN, AH Liver cheese]","[0.89, NaN, NaN, 1.15]","[2 stuks, NaN, NaN, 150 g]","[1.0, 621.0435, 310.52175, 177.441, 118.294]","['stuk', 'ml', 'ml', 'ml', 'ml']","[2.0, 1.0, 1.0, 150.0]","['stuk', 'NaN', 'NaN', 'g']","[0.89, 0.0, 0.0, 1.15]",1.805381
4,4,Reeses Cups(Candy),"[236.588 ml peanut butter, 177.441 ml graham cracker crumbs, 236.588 ml melted butter, 453.592 grams 828.058 ml powdered sugar, 1.0 large pkg. chocolate chips]","[""peanut butter"", ""graham cracker crumbs"", ""butter"", ""powdered sugar"", ""chocolate chips""]","['peanut butter', 'rum', 'butter', 'powdered sugar', 'chocolate']","[AH Creamy peanut butter, Drum Blue, AH Herb butter, AH Egg waffles with powdered sugar, AH Hot chocolate]","[2.39, 14.3, 1.25, 1.19, 2.39]","[350 g, 40 g, 100 g, 6 stuks, 8 stuks]","[236.588, 177.441, 236.588, 453.592, 1.0]","['ml', 'ml', 'ml', 'g', 'stuk']","[350.0, 40.0, 100.0, 6.0, 8.0]","['g', 'g', 'g', 'stuk', 'stuk']","[2.39, 14.3, 1.25, 1.19, 2.39]",158.269229
5,5,Cheeseburger Potato Soup,"[6.0 baking potatoes, 453.592 grams of extra lean ground beef, 157.7253333333333 ml butter or margarine, 1419.528 ml milk, 3.69669 ml salt, 2.46446 ml pepper, 354.882 ml 177.441 ml shredded cheddar cheese, divided, 12.0 sliced bacon, cooked, crumbled and divided, 4.0 green onion, chopped and divided, 236.588 ml carton sour cream optional]","[""baking p